Задание 1.
Обучите нейронную сеть решать шифр Цезаря.

Что необходимо сделать:

Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
Сделать нейронную сеть
Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
Проверить качество

Задание 2.
Выполнить практическую работу из лекционного ноутбука.

Построить RNN-ячейку на основе полносвязных слоев
Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [ ]:
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети
import re

In [ ]:
import pandas as pd
df = pd.read_csv('Constraint_Train.csv')
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
# предобработка данных
def remove_punctuation(text):
    return re.sub(r'[^A-Za-z\s]', '', text.lower())


df['cleaned_text'] = df['tweet'].apply(remove_punctuation)
df.head()

,id,tweet,label,cleaned_text
0,1,The CDC currently reports 99031 deaths. In gen...,real,the cdc currently reports deaths in general t...
1,2,States reported 1121 deaths a small rise from ...,real,states reported deaths a small rise from last...
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,politically correct woman almost uses pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,indiafightscorona we have covid testing labor...
4,5,Populous states can generate large case counts...,real,populous states can generate large case counts...


In [ ]:
words = df['cleaned_text'].str.cat(sep=' ').split()
words[0:10]

['the',
 'cdc',
 'currently',
 'reports',
 'deaths',
 'in',
 'general',
 'the',
 'discrepancies',
 'in']

In [ ]:
alphabet ='abcdefghijklmnopqrstuvwxyz'
def find_position(ch, alphabet):

  return alphabet.find(ch)


def shift_position(word):
  shift = 2
  shift_word =''
  for ch in word:
    curr_pos = find_position(ch, alphabet)

    if curr_pos >=len(alphabet)-shift:
      curr_pos-=len(alphabet)
    shift_word+= shift_word.join(alphabet[curr_pos+shift])
  return shift_word

shift_position('xyz')


'zab'

In [ ]:
target = []

def shift_words(words):
  out =[]
  for w in words:
    out.append(shift_position(w))
  return out

target = shift_words(words)
target[:10]

['vjg',
 'efe',
 'ewttgpvna',
 'tgrqtvu',
 'fgcvju',
 'kp',
 'igpgtcn',
 'vjg',
 'fkuetgrcpekgu',
 'kp']

In [ ]:
len(target)

166161

In [ ]:

MAX_LEN = max(len(s) for s in target)
MAX_LEN

61

In [ ]:
# необходимо закодировать их в числа

In [ ]:
CHARS = 'abcdefghijklmnopqrstuvwxyz '  # все символы, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс

In [ ]:
CHAR_TO_INDEX

{'none': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 ' ': 27}

In [ ]:
INDEX_TO_CHAR

['none',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 ' ']

In [ ]:
len(INDEX_TO_CHAR)

28

In [ ]:
X = torch.zeros((len(words), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(words)):  # для каждого предложения
    for j, w in enumerate(words[i]):  # для каждого токена
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [ ]:
X[0:5] # закодировали слова в векторы

tensor([[20,  8,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0],
        [ 3,  4,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0],
        [ 3, 21, 18, 18,  5, 14, 20, 12, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0],
        [18,  5, 16, 15, 18, 20, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [ ]:
embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)

In [ ]:
Y = torch.zeros((len(target), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(target)):  # для каждого предложения
    for j, w in enumerate(target[i]):  # для каждого токена
        Y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [ ]:
Y[0:5]

tensor([[22, 10,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0],
        [ 5,  6,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0],
        [ 5, 23, 20, 20,  7, 16, 22, 14,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0],
        [20,  7, 18, 17, 20, 22, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = torch.nn.RNN(30, 128)
        self.out = torch.nn.Linear(128, 28)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences) # берем предложения
        x, s = self.rnn(x) # берём выход с последнего слоя для всех токенов, а не скрытое состояние (рекурентно)
        return self.out(x)

In [ ]:
model = Network()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [ ]:
 # Проверка доступности CUDA
if torch.cuda.is_available():
  device = torch.device("cuda")  # Использовать GPU
  print("CUDA доступна. Используем GPU.")
else:
  device = torch.device("cpu")  # Использовать CPU
  print("CUDA недоступна. Используем CPU.")






CUDA доступна. Используем GPU.


In [ ]:
for ep in range(20):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 500)):
        # берём батч в 100 элементов
        x_batch = X[i * 500:(i + 1) * 500]
        y_batch = Y[i * 500:(i + 1) * 500]
        X_batch = x_batch[:, :-1]
        Y_batch = y_batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 45.749, Train loss: 0.316
Epoch 1. Time: 42.549, Train loss: 0.259
Epoch 2. Time: 44.638, Train loss: 0.247
Epoch 3. Time: 43.275, Train loss: 0.241
Epoch 4. Time: 42.063, Train loss: 0.237
Epoch 5. Time: 44.827, Train loss: 0.235
Epoch 6. Time: 43.500, Train loss: 0.232
Epoch 7. Time: 44.889, Train loss: 0.230
Epoch 8. Time: 44.106, Train loss: 0.228
Epoch 9. Time: 42.399, Train loss: 0.227
Epoch 10. Time: 43.524, Train loss: 0.226
Epoch 11. Time: 44.213, Train loss: 0.225
Epoch 12. Time: 43.542, Train loss: 0.223
Epoch 13. Time: 43.427, Train loss: 0.223
Epoch 14. Time: 45.643, Train loss: 0.222
Epoch 15. Time: 45.209, Train loss: 0.221
Epoch 16. Time: 43.699, Train loss: 0.220
Epoch 17. Time: 43.609, Train loss: 0.219
Epoch 18. Time: 43.746, Train loss: 0.219
Epoch 19. Time: 43.324, Train loss: 0.218


In [ ]:
X[3:4]#начальное значение не закодированное

tensor([[18,  5, 16, 15, 18, 20, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0]])

In [ ]:
answers = model.forward(X[3:4]) # то что загоняем на вход
probas, indices = answers.topk(1) # то что загоняем на выход

In [ ]:
indices.flatten() # предсказываемое значение после кодировки

tensor([ 7,  6, 21, 16,  7, 10, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0])

In [ ]:
# Метод перевода вектора в слова
def tensor_to_string (tensor):

  out =''
  data = tensor.tolist()
  for d in data:
    for i in range(1,len(INDEX_TO_CHAR)):
      if d==i:
        out+= out.join(INDEX_TO_CHAR[i])
        break
  return out

In [ ]:
 test = tensor_to_string(indices.flatten())
 target = tensor_to_string(X[3:4].flatten())
 shift_target = shift_words(target)
 target, ''.join(shift_target), test

('reports', 'tgrqtvu', 'gfupgjv')